<h1 style="text-align:center">NBI SAS Output</h1>

###### The current process is to generate output for NBI Table shell

**Instruction**
* Copy the original SAS dataset first
* Import SAS dataset
* Process the code and it generates a excel file which resembles to the table shell

In [113]:
# ----------------
# import libraries
# ----------------

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
import datetime
from shutil import copyfile

In [111]:
# ------------------
# import SAS dataset
# ------------------
# Please note: the import file can also be a csv file which is converted from the SAS dataset
# PLease note: pandas may truncate infinite decimals which might cause the output accuracy might be slightly different than the output from SAS

file_path = '//prod-isilon.corporate.ivh/SASDataHome/SASProduction/Boehringer Ingelheim/EMPRISE/SASData/Empa_DPP4_main/'
# doing the following is not suggest as it's processing on the orignal file.
#test_df = pd.read_sas(file_path+'patpop_ps_add.sas7bdat')
dest_path = 'C:/Users/guzhao/Documents/GitHub/Code_Generator_SAS/Code-Generator-SAS/'
# uncomment this to copy the file to the juypter folder
#copyfile(file_path+"patpop_ps_add.sas7bdat", dest_path+'patpop_ps_add.sas7bdat')

#df = pd.read_sas('patpop_baseline_v1.sas7bdat')
#df = pd.read_sas('patpop_ps.sas7bdat')
#df = pd.read_sas('patpop_baseline_add.sas7bdat')
df = pd.read_sas('patpop_ps_add.sas7bdat')

In [48]:
# ---------------------
# create data etl rules
# ---------------------

def age_group(age):
    # create age bin
    if age >= 18 and age <= 54:
        return '18-54'
    elif age >= 55 and age <= 64:
        return '55-64'
    elif age >= 65 and age <= 74:
        return '65-74'
    else:
        return '75+'

def sex(x):
    # create gender labels
    if x == 1:
        return 'male'
    elif x == 2:
        return 'female'
    else:
        return 'missing'

def indicator(x):
    # create indicator labels for numeric indicator features
    if x == 0:
        return 'No'
    elif x == 1:
        return 'Yes'
    else:
        return 'Missing'
    
def dot(x):
    # convert missing values from 0 to .
    if x == 0:
        return '.'
    else:
        return x

In [49]:
# --------------------
# apply data etl rules
# --------------------

# create feature age_group
df['age_group'] = df['age'].apply(lambda x: age_group(x))
# create feature sex_name
df['sex_name'] = df['sex'].apply(lambda x: sex(x))
# create feature c_year to get calendar year from index date
df['c_year'] = df['index_dt'].apply(lambda x: x.to_pydatetime().year)

# use below when using csv file to create c_year
#df['c_year'] = df['index_dt'].astype(str).str[:4]

In [50]:
# ----------------------------------------------
# create output tables for categorical variables
# ----------------------------------------------

def cross(ind, col):
    '''
    purpose: return contigency tables for categorical features
    
    variable:
        ind - the index for the contigency table
        col - the column for the contigency table
    ''' 
    return pd.crosstab(index=ind, columns=df[col])

def char_std(d, ind):
    '''
    purpose: return std for categoical features
    
    variable:
        d - the dataframe needs to be used for calculation
        ind - the index for the output data frame which is used for merging back to the contigency table
    '''
    # create p1 and p2 for std calculation
    p1 = d.iloc[0,0]/cohort_tab.iloc[0,0]
    p2 = d.iloc[0,1]/cohort_tab.iloc[0,1]
    
    # calculate std
    if p1 == 0 and p2 == 0:
        _std_data = {'std': '.'}
    else:
        _std_data = {'std': np.absolute(p1 - p2)/np.sqrt((p1*(1-p1) + p2*(1-p2))/2)}
    
    return d.join(pd.DataFrame(_std_data, index=[ind])) 

In [51]:
# ------------------------------------------
# create output tables for numeric variables
# ------------------------------------------


def numeric_tab(var, item, miss_trigger):
    '''
    purpose: return descriptive tables for numerical features
    
    variable:
        var - the feature needs to be processed
        item - the main data frame that has been grouped by cohort
        miss_trigger - the trigger for output number of missing values
    '''
    # define each descriptive statistics
    _mea = item.mean().round(2).to_frame().rename(columns={var:var+'_mean'}).transpose()
    _s = item.std().round(2).to_frame().rename(columns={var:var+'_std'}).transpose()
    _med = item.median().round(2).to_frame().rename(columns={var:var+'_median'}).transpose()
    _q25 = item.quantile(0.25,interpolation='midpoint').round(2).to_frame()
    _q75 = item.quantile(0.75,interpolation='midpoint').round(2).to_frame()
    _min = item.min().round(2).to_frame()
    _max = item.max().round(2).to_frame()
    _nul = item.apply(lambda x: x.isnull().sum()).apply(lambda x: dot(x)).to_frame().rename(columns={var:var+'_missing'}).transpose()
        
    # combine quantiles
    _q = _q25.rename(columns={var:var+'_Q1'}).join(_q75.rename(columns={var: var+'_Q3'}))
    _q[var+'_Q1_Q3'] = _q[var+'_Q1'].map(str) + '; ' + _q[var+'_Q3'].map(str)
    _q = _q.drop([var+'_Q1', var+'_Q3'], axis=1).transpose()
    
    # combine min and max
    _m = _min.rename(columns={var:var+'_min'}).join(_max.rename(columns={var: var+'_max'}))
    _m[var+'_min_max'] = _m[var+'_min'].map(str) + '; ' + _m[var+'_max'].map(str)
    _m = _m.drop([var+'_min', var+'_max'], axis=1).transpose()

    # calculate std
    _std_data = {'std': np.absolute(_mea.iloc[0,0] - _mea.iloc[0,1])/np.sqrt((np.square(_s.iloc[0,0]) + np.square(_s.iloc[0,1]))/2)}
    _std = pd.DataFrame(_std_data, index=[var+'_mean'])
    
    # control whether to output number of missing values or not
    if miss_trigger == 1:
        _df = pd.concat([_mea, _s, _med, _q, _m, _nul])
    else:
        _df = pd.concat([_mea, _s, _med, _q, _m])
    
    return _df.join(_std.round(3)).fillna('NA')

In [58]:
# ------------------------------------
# acquire SAS variables for processing 
# ------------------------------------
# this excel file may require proper changes before it can be processed and the rules are as follows:
# 1.Record a MACRO for what I'm going to do
# 2.Match each tab with corresponding output tables, create new tab if anything's not in the output table like other drugs
# 3.Make sure all the variables names are filled in order to output NA correctly
# 4.Make sure the output sequence matches
# 5.Create type variable to control categorical or numerical output where C stands for categorical and N stands for Numerical

sas_lifestyle = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='life_style') # not use this at this moment
sas_Diabetes_complications = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='Diabetes complications')
sas_other_comorbidity = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='other_comorbidity')
sas_other_drugs = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='other_drugs')
sas_laboratory = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='laboratory')
sas_Prior_concomitant = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='Prior concomitant')
sas_resource_utilization = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='resource_utilization')
sas_cost = pd.read_excel('NBI covariates name 1 .xlsx', sheet_name='cost')


sas_dummy = pd.DataFrame({'Covariates in table shells ':[''], 'Variable_name':[''], 'Type':[''], 'Style':['']}, index=['placeholder'])

sas_var = pd.concat([sas_dummy, sas_lifestyle, sas_dummy, sas_Diabetes_complications, sas_dummy, 
                     sas_other_comorbidity, sas_dummy, sas_laboratory, sas_dummy,
                     sas_Prior_concomitant, sas_dummy, sas_other_drugs, sas_dummy,
                     sas_resource_utilization, sas_dummy, sas_cost])

In [97]:
# ----------------------
# generate output tables
# ----------------------
# strategy: 
#    for categorical tables, a dummy data frame will be createed first with index as 'Yes', 'No' and 'Missing', then the output values will be filled in
#    for numerical tables, the output from the function numeric_tab is immediately usable


# prepare excel writer
writer = pd.ExcelWriter('NBI_results_psa.xlsx', engine='xlsxwriter')

# generate "Baseline demographics"
cohort_tab = cross('number of patients', 'cohort')
c_tab = char_std(cohort_tab, 'number of patients')
age_all = numeric_tab('age', df.groupby(['cohort']).age, 0)
age_tab = cross(df['age_group'], 'cohort')
sex_tab = char_std(cross(df['sex_name'], 'cohort'), 'female')
race_dummy = pd.DataFrame({1.0:['', '', '', ''], 2.0:['', '', '', ''], 'std':['', '', '', '']}, index=['race_Category_1', 'race_Category_2', 'race_Category_3', 'race_Missing'])
soc_dummy = pd.DataFrame({1.0:['', '', '', ''], 2.0:['', '', '', ''], 'std':['', '', '', '']}, index=['soc_Low', 'soc_Intermediate', 'soc_High', 'soc_Missing'])
year_dummy = pd.DataFrame({1.0:[0, 0, 0, 0, 0, 0, 0], 2.0:[0, 0, 0, 0, 0, 0, 0], 'std':['.', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']}, index=[2012, 2013, 2014, 2015, 2016, 2017, 2018])
year_tab = cross(df['c_year'], 'cohort')
year_dummy.update(year_tab)

# generate "the rest of the output"
output=[]
for value, t, s in zip(sas_var['Variable_name'], sas_var['Type'], sas_var['Style']):
    if value in df.columns and t == 'C':   
        
        if s == 'F':
            dummy_df = pd.DataFrame({1.0:[0, 0, '.'], 2.0:[0, 0, '.'], 'std':['.', 'NA', 'NA']}, index=['Yes', 'No', 'Missing'])
            value_tab = char_std(cross(df[value].apply(lambda x:indicator(x)), 'cohort').iloc[::-1], 'Yes')
            new_index = pd.DataFrame({'new_index':[value+'_Yes', value+'_No', value+'_Missing']}, index=['Yes', 'No', 'Missing'])
        elif s == 'Y':
            dummy_df = pd.DataFrame({1.0:[0], 2.0:[0], 'std':['.']}, index=['Yes'])
            value_tab = char_std(cross(df[value].apply(lambda x:indicator(x)), 'cohort').iloc[::-1], 'Yes')
            if ('Yes' in value_tab.index.values.tolist()):
                value_tab = value_tab.loc[['Yes']]
            else:
                value_tab = pd.DataFrame({1.0:[0], 2.0:[0], 'std':['.']}, index=[value+'_Yes'])
            new_index = pd.DataFrame({'new_index':[value+'_Yes']}, index=['Yes'])
        
        dummy_df.update(value_tab)
        dummy_df.index.name = value   
        dummy_df = dummy_df.join(new_index)                        
        dummy_df = dummy_df.reset_index(drop=True)
        dummy_df = dummy_df.set_index('new_index')    
        output.append(dummy_df) 
    elif value in df.columns and t == 'N' and s == 'Y':
        df_g = df.groupby(['cohort'])
        value_tab = numeric_tab(value, df_g[value], 1)
        output.append(value_tab)
    elif value in df.columns and t == 'N' and s == 'N':
        df_g = df.groupby(['cohort'])
        value_tab = numeric_tab(value, df_g[value], 0)
        output.append(value_tab)
    elif value not in df.columns and t == 'C':
        if s == 'F':
            value_tab = pd.DataFrame({1.0:['', '', ''], 2.0:['', '', ''], 'std':['', 'NA', 'NA']}, index=[value+'_Yes', value+'_No', value+'_Missing'])
        elif s == 'Y':
            value_tab = pd.DataFrame({1.0:[''], 2.0:[''], 'std':['']}, index=[value+'_Yes'])

        value_tab.index.name = value      
        output.append(value_tab)
    elif value not in df.columns and t == 'N':
        value_tab = pd.DataFrame({1.0:['', '', '', '', '', ''], 2.0:['', '', '', '', '', ''], 'std':['', 'NA', 'NA', 'NA', 'NA', 'NA']}, index=[value+'_mean', value+'_std', value+'_median', value+'_Q1_Q3', value+'_min_max', value+'_missing'])
        value_tab.index.name = value    
        output.append(value_tab)
    else:
        value_tab = pd.DataFrame({1.0:[''], 2.0:[''], 'std':['']}, index=[value+'_placeholder'])
        value_tab.index.name = value    
        output.append(value_tab)

output = pd.concat(output)        
_output = pd.concat([c_tab, age_all, age_tab, sex_tab, race_dummy, soc_dummy, year_dummy, output])
_output = _output.replace(np.nan, 'NA', regex=True)
_output.to_excel(writer, sheet_name='output')

writer.save()

C:\Users\guzhao\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
